본 강좌에서는 컨볼루션 신경망 모델의 성능을 높이기 위한 방법 중 하나인 데이터를 부풀리기에 대해서 알아보겠습니다. 

1. 현실적인 문제
1. 기존 모델 결과보기
1. 데이터 부풀리기
1. 개선 모델 결과보기

---

### 현실적인 문제

[컨볼루션 신경망 모델 만들어보기](https://tykimos.github.io/Keras/2017/03/08/CNN_Getting_Started/) 강좌에서 사용하였던 원, 사각형, 삼각형 데이터셋을 예제로 살펴보겠습니다. 구성은 훈련셋과 시험셋으로 되어 있는 데, 아래 그림은 훈련셋입니다.

#### 훈련셋
![data](http://tykimos.github.com/Keras/warehouse/2017-3-8-CNN_Data_Augmentation_1.png)

그리고 아래 그림은 시험셋입니다. 훈련셋과 시험셋은 모두 한사람(제가) 그린 것이라 거의 비슷합니다. 그래서 그런지 100% 정확도의 좋은 결과를 얻었나 봅니다.

#### 시험셋
![data](http://tykimos.github.com/Keras/warehouse/2017-3-8-CNN_Data_Augmentation_2.png)

100% 정확도를 얻은 모델이니 원, 사각형, 삼각형을 그려주면 다 분류를 해보겠다며 지인(아내)에게 자랑을 해봅니다. 그래서 지인이 그려준 시험셋은 다음과 같습니다.

#### 도전 시험셋
![data](http://tykimos.github.com/Keras/warehouse/2017-3-8-CNN_Data_Augmentation_3.png)

막상 시험셋을 받아보니 자신감이 없어지면서 여러가지 생각이 듭니다.

- 아, 이것도 원, 사각형, 삼각형이구나
- 왜 이런 데이터를 진작에 학습시키지 않았을까?
- 새로 받은 시험셋 일부를 학습시켜볼까?
- 이렇게 간단한 문제도 개발과 현실과의 차이가 이렇게 나는데, 실제 문제는 더 상황이 좋지 않겠지?

결국 하나의 결론에 도달합니다.

    개발자가 시험셋을 만들면 안된다.

하지만 어떠한 문제에서도 미래에 들어올 데이터에 대해서는 알 수가 없기 때문에, 비슷한 상황일 것 같습니다. 먼저 도전 시험셋으로 시험한 결과를 살펴본 뒤, 한정적인 훈련셋을 이용하여 최대한 발생할 수 있는 경우을 고려하여 훈련셋을 만드는 방법인 `데이터 부풀리기`에 대해서 알아보겠습니다.

---

### 기존 모델 결과보기

[컨볼루션 신경망 모델 만들어보기](https://tykimos.github.io/Keras/2017/03/08/CNN_Getting_Started/) 강좌에서 사용했던 모델을 그대로 가지고 왔습니다. 제가 만든 시험셋에서는 결과가 100%나왔는데, 도전 시험셋으론 어떤 결과가 나오는 지 테스트해보겠습니다.

In [3]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(3)

from keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'warehouse/hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'warehouse/hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

# 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=45,
        epochs=200,
        validation_data=test_generator,
        validation_steps=15)

# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            test_generator, 
            steps = 15)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")

output = model.predict_generator(
            test_generator, 
            steps = 15)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/200
45/45 [==============================] - 0s - loss: 0.3770 - acc: 0.8593 - val_loss: 4.3110 - val_acc: 0.4000
Epoch 2/200
45/45 [==============================] - 0s - loss: 0.0031 - acc: 1.0000 - val_loss: 5.4077 - val_acc: 0.4000
Epoch 3/200
45/45 [==============================] - 0s - loss: 2.0370e-04 - acc: 1.0000 - val_loss: 5.6962 - val_acc: 0.4000
Epoch 4/200
45/45 [==============================] - 0s - loss: 1.3430e-04 - acc: 1.0000 - val_loss: 5.9781 - val_acc: 0.3778
Epoch 5/200
45/45 [==============================] - 0s - loss: 1.0009e-04 - acc: 1.0000 - val_loss: 6.2118 - val_acc: 0.3778
Epoch 6/200
45/45 [==============================] - 0s - loss: 7.5384e-05 - acc: 1.0000 - val_loss: 6.5347 - val_acc: 0.3556
Epoch 7/200
45/45 [==============================] - 0s - loss: 5.6719e-05 - acc: 1.0000 - val_loss: 6.5132 - val_acc: 0.4000
Epoch 8/200
45/45 [===========================

---

### 데이터 부풀리기

데이터를 어떻게 부풀데이터 부풀리는 방법 설명

![data](http://tykimos.github.com/Keras/warehouse/2017-3-8-CNN_Data_Augmentation_4.png)

- rotation_range : 임의의 회전 각도 범위이며 값은 도(정수)을 나타냅니다.
- width_shift_range : 수평방향으로 임의로 움직이는 범위이며 값은 전체 넓이의 비율(실수)을 나타냅니다. 너비가 100픽셀일 때, 10%이면, 10만큼 임의로 이동합니다.
- height_shift_range : 수직방향으로 임의로 움직이는 범위이며 값은 전체 넓이의 비율(실수)을 나타냅니다. 너비가 100픽셀일 때, 10%이면, 10만큼 임의로 이동합니다.
- shear_range : Shear Intensity (Shear angle in counter-clockwise direction as radians)
- zoom_range :  Float or [lower, upper]. Range for random zoom. If a float,  [lower, upper] = [1-zoom_range, 1+zoom_range].
- horizontal_flip : 
- vertical_flip : 
- fill_mode : 


featurewise_center: Boolean. Set input mean to 0 over the dataset, feature-wise.
samplewise_center: Boolean. Set each sample mean to 0.
featurewise_std_normalization: Boolean. Divide inputs by std of the dataset, feature-wise.
samplewise_std_normalization: Boolean. Divide each input by its std.
zca_whitening: Boolean. Apply ZCA whitening.
rotation_range: Int. Degree range for random rotations.
width_shift_range: Float (fraction of total width). Range for random horizontal shifts.
height_shift_range: Float (fraction of total height). Range for random vertical shifts.
shear_range: Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
zoom_range: Float or [lower, upper]. Range for random zoom. If a float,  [lower, upper] = [1-zoom_range, 1+zoom_range].
channel_shift_range: Float. Range for random channel shifts.
fill_mode: One of {"constant", "nearest", "reflect" or "wrap"}. Points outside the boundaries of the input are filled according to the given mode.
cval: Float or Int. Value used for points outside the boundaries when fill_mode = "constant".
horizontal_flip: Boolean. Randomly flip inputs horizontally.
vertical_flip: Boolean. Randomly flip inputs vertically.
rescale: rescaling factor. Defaults to None. If None or 0, no rescaling is applied, otherwise we multiply the data by the value provided (before applying any other transformation).
preprocessing_function: function that will be implied on each input. The function will run before any other modification on it. The function should take one argument: one image (Numpy tensor with rank 3), and should output a Numpy tensor with the same shape.
data_format: One of {"channels_first", "channels_last"}. "channels_last" mode means that the images should have shape  (samples, height, width, channels), "channels_first" mode means that the images should have shape  (samples, channels, height, width). It defaults to the image_data_format value found in your Keras config file at ~/.keras/keras.json. If you never set it, then it will be "channels_last".


#### rotation_range = 90
![data](http://tykimos.github.com/Keras/warehouse/2017-3-8-CNN_Data_Augmentation_5_rotate.png)

In [30]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(5)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   zca_whitening=True,
                                   fill_mode='nearest')
                                   
img = load_img('warehouse/hard_handwriting_shape/train/triangle/triangle001.png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir='warehouse/preview', save_prefix='tri', save_format='png'):
    i += 1
    if i > 8:
       break  # otherwise the generator would loop indefinitely

/Users/tykimos/Projects/Keras/venv/lib/python2.7/site-packages/keras/preprocessing/image.py:518: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [60]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(5)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')
                                   
img = load_img('warehouse/hard_handwriting_shape/train/triangle/triangle001.png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir='warehouse/preview', save_prefix='tri', save_format='png'):
    i += 1
    if i > 30:
        break  # otherwise the generator would loop indefinitely

In [59]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(3)

from keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.4,
                                   zoom_range=[-0.5, 0.5],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'warehouse/hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'warehouse/hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.layers import Dropout

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=45,
        epochs=200,
        validation_data=test_generator,
        validation_steps=15)

# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            test_generator, 
            steps = 15)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")

output = model.predict_generator(
            test_generator, 
            steps = 15)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/200
45/45 [==============================] - 0s - loss: 1.0008 - acc: 0.4741 - val_loss: 2.6793 - val_acc: 0.3333
Epoch 2/200
45/45 [==============================] - 0s - loss: 0.9786 - acc: 0.4667 - val_loss: 2.6156 - val_acc: 0.3111
Epoch 3/200
45/45 [==============================] - 0s - loss: 0.9532 - acc: 0.4444 - val_loss: 2.7727 - val_acc: 0.3778
Epoch 4/200
45/45 [==============================] - 0s - loss: 0.7783 - acc: 0.5704 - val_loss: 4.5869 - val_acc: 0.3333
Epoch 5/200
45/45 [==============================] - 0s - loss: 0.8642 - acc: 0.5037 - val_loss: 4.6654 - val_acc: 0.3333
Epoch 6/200
45/45 [==============================] - 0s - loss: 0.8593 - acc: 0.5111 - val_loss: 6.0541 - val_acc: 0.3111
Epoch 7/200
45/45 [==============================] - 0s - loss: 0.8762 - acc: 0.5111 - val_loss: 3.9737 - val_acc: 0.3111
Epoch 8/200
45/45 [==============================] - 0s - loss: 0.

KeyboardInterrupt: 

In [39]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(3)

from keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zca_whitening = True,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.9,
                                   zoom_range=0.5,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'warehouse/hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'warehouse/hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.layers import Dropout

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=45,
        epochs=200,
        validation_data=test_generator,
        validation_steps=15)

# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            test_generator, 
            steps = 15)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")

output = model.predict_generator(
            test_generator, 
            steps = 15)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/200
45/45 [==============================] - 0s - loss: 1.1503 - acc: 0.3704 - val_loss: 1.1086 - val_acc: 0.2667
Epoch 2/200
45/45 [==============================] - 0s - loss: 1.0799 - acc: 0.4370 - val_loss: 1.1284 - val_acc: 0.2667
Epoch 3/200
45/45 [==============================] - 0s - loss: 0.9941 - acc: 0.4963 - val_loss: 1.1966 - val_acc: 0.3111
Epoch 4/200
45/45 [==============================] - 0s - loss: 0.9280 - acc: 0.4519 - val_loss: 1.1636 - val_acc: 0.3333
Epoch 5/200
45/45 [==============================] - 0s - loss: 0.9974 - acc: 0.4444 - val_loss: 1.2079 - val_acc: 0.2889
Epoch 6/200
45/45 [==============================] - 0s - loss: 0.9910 - acc: 0.5556 - val_loss: 1.2824 - val_acc: 0.2667
Epoch 7/200
45/45 [==============================] - 1s - loss: 0.9379 - acc: 0.5407 - val_loss: 1.3094 - val_acc: 0.4000
Epoch 8/200
45/45 [==============================] - 0s - loss: 0.

/Users/tykimos/Projects/Keras/venv/lib/python2.7/site-packages/keras/preprocessing/image.py:518: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


입력되는 패치 이미지는 다음과 같습니다.

![data](http://tykimos.github.com/Keras/warehouse/2017-3-8-CNN_Data_Augmentation_1.png)

위의 패치 이미지를 입력으로 데이터 부풀리기를 한 결과는 다음과 같습니다.

![data](http://tykimos.github.com/Keras/warehouse/2017-3-8-CNN_Data_Augmentation_2.png)

---

### 문제 정의하기

좋은 예제와 그와 관련된 데이터셋도 공개된 것이 많이 있지만, 직접 문제를 정의하고 데이터를 만들어보는 것도 처럼 딥러닝을 접하시는 분들에게는 크게 도움이 될 것 같습니다. 컨볼루션 신경망 모델에 적합한 문제는 이미지 기반의 분류입니다. 따라서 우리는 직접 손으로 삼각형, 사각형, 원을 그려 이미지로 저장한 다음 이를 분류해보는 모델을 만들어보겠습니다. 문제 형태와 입출력을 다음과 같이 정의해봅니다.
* 문제 형태 : 다중 클래스 분류
* 입력 : 손으로 그린 삼각형, 사각형, 원 이미지
* 출력 : 삼각형, 사각형, 원일 확률을 나타내는 벡터

매번 실행 시마다 결과가 달라지지 않도록 랜덤 시드를 명시적으로 지정합니다.

In [ ]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(5)

---

### 데이터셋 준비하기

손으로 그린 삼각형, 사각형, 원 이미지를 만들기 위해서는 여러가지 방법이 있을 수 있겠네요. 테블릿을 이용할 수도 있고, 종이에 그려서 사진으로 찍을 수도 있습니다. 저는 그림 그리는 툴을 이용해서 만들어봤습니다. 이미지 사이즈는 24 x 24 정도로 해봤습니다. 

![data](http://tykimos.github.com/Keras/warehouse/2017-3-8_CNN_Getting_Started_1.png)

모양별로 20개 정도를 만들어서 15개를 훈련에 사용하고, 5개를 테스트에 사용해보겠습니다. 이미지는 png나 jpg로 저장합니다. 실제로 데이터셋이 어떻게 구성되어 있는 지 모른 체 튜토리얼을 따라하거나 예제 코드를 실행시키다보면 결과는 잘 나오지만 막상 실제 문제에 적용할 때 막막해질 때가 있습니다. 간단한 예제로 직접 데이터셋을 만들어봄으로써 실제 문제에 접근할 때 시행착오를 줄이는 것이 중요합니다.

데이터셋 폴더는 다음과 같이 구성했습니다.

- train
 - circle
 - rectangle
 - triangle
- validation
 - circle
 - rectangle
 - triangle
 
![data](http://tykimos.github.com/Keras/warehouse/2017-3-8_CNN_Getting_Started_2.png)

---

### 데이터셋 불러오기

케라스에서는 이미지 파일을 쉽게 학습시킬 수 있도록 ImageDataGenerator 클래스를 제공합니다. ImageDataGenerator 클래스는 데이터 증강 (data augmentation)을 위해 막강한 기능을 제공하는 데, 이 기능들은 다른 강좌에세 다루기로 하고, 본 강좌에서는 특정 폴더에 이미지를 분류 해놓았을 때 이를 학습시키기 위한 데이터셋으로 만들어주는 기능을 사용해보겠습니다.

먼저 ImageDataGenerator 클래스를 이용하여 객체를 생성한 뒤 flow_from_directory() 함수를 호출하여 제네레이터(generator)를 생성합니다. flow_from_directory() 함수의 주요인자는 다음과 같습니다.

- 첫번재 인자 : 이미지 경로를 지정합니다.
- target_size : 패치 이미지 크기를 지정합니다. 폴더에 있는 원본 이미지 크기가 다르더라도 target_size에 지정된 크기로 자동 조절됩니다.
- batch_size : 배치 크기를 지정합니다.
- class_mode : 분류 방식에 대해서 지정합니다.
    - categorical : 2D one-hot 부호화된 라벨이 반환됩니다.
    - binary : 1D 이진 라벨이 반환됩니다.
    - sparse : 1D 정수 라벨이 반환됩니다.
    - None : 라벨이 반환되지 않습니다.

본 예제에서는 패치 이미지 크기를 24 x 24로 하였으니 target_size도 (24, 24)로 셋팅하였습니다. 훈련 데이터 수가 클래스당 15개이니 배치 크기를 3으로 지정하여 총 5번 배치를 수행하면 하나의 epoch가 수행될 수 있도록 하였습니다. 다중 클래스 문제이므로 class_mode는 'categorical'로 지정하였습니다. 그리고 제네레이터는 훈련용과 검증용으로 두 개를 만들었습니다. 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'warehouse/handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
        'warehouse/handwriting_shape/validation',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

---

### 모델 구성하기

영상 분류에 높은 성능을 보이고 있는 컨볼루션 신경망 모델을 구성해보겠습니다. 이전 강좌에서 만들어봤던 모델 위에 입력을 24 x 24, 3개 채널 이미지를 받을 수 있고 필터를 12개를 가진 컨볼루션 레이어와 맥스풀링 레이어를 추가해봤습니다.

* 컨볼루션 레이어 : 입력 이미지 크기 24 x 24, 입력 이미지 채널 3개, 필터 크기 3 x 3, 필터 수 12개, 경계 타입 'same', 활성화 함수 'relu'
* 맥스풀링 레이어 : 풀 크기 2 x 2
* 컨볼루션 레이어 : 입력 이미지 크기 8 x 8, 입력 이미지 채널 1개, 필터 크기 3 x 3, 필터 수 2개, 경계 타입 'same', 활성화 함수 'relu'
* 맥스풀링 레이어 : 풀 크기 2 x 2
* 컨볼루션 레이어 : 입력 이미지 크기 4 x 4, 입력 이미지 채널 2개, 필터 크기 2 x 2, 필터 수 3개, 경계 타입 'same', 활성화 함수 'relu'
* 맥스풀링 레이어 : 풀 크기 2 x 2
* 플래튼 레이어
* 댄스 레이어 : 입력 뉴런 수 12개, 출력 뉴런 수 8개, 활성화 함수 'relu'
* 댄스 레이어 : 입력 뉴런 수 8개, 출력 뉴런 수 3개, 활성화 함수 'softmax'

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

# 모델 구성하기
model = Sequential()
model.add(Convolution2D(12, 3, 3, border_mode='same', input_shape=(3, 24, 24), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(2, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(3, 2, 2, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

![model](http://tykimos.github.com/Keras/warehouse/2017-3-8_CNN_Getting_Started_3.png)

---

### 모델 엮기

모델을 정의했다면 모델을 손실함수와 최적화 알고리즘으로 엮어봅니다. 
- loss : 현재 가중치 세트를 평가하는 데 사용한 손실 함수 입니다. 다중 클래스 문제이므로 'categorical_crossentropy'으로 지정합니다.
- optimizer : 최적의 가중치를 검색하는 데 사용되는 최적화 알고리즘으로 효율적인 경사 하강법 알고리즘 중 하나인 'adam'을 사용합니다.
- metrics : 평가 척도를 나타내며 분류 문제에서는 일반적으로 'accuracy'으로 지정합니다.

In [ ]:
# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습시키기

케라스에서는 모델을 학습시킬 때 주로 fit 함수를 사용하지만 제네레이터로 생성된 배치로 학습시킬 경우에는 fit_generator 함수를 사용합니다. 본 예제에서는 ImageDataGenerator라는 제네레이터로 이미지를 담고 있는 배치로 학습시키기 때문에 fit_generator 함수를 사용하겠습니다.

- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 train_generator으로 지정합니다.
- samples_per_epoch : 한 epoch에 사용한 샘플 수를 지정합니다. 총 45개의 훈련 샘플이 있으므로 45로 지정합니다.
- nb_epoch : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정합니다. 100번을 반복적으로 학습시켜 보겠습니다.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정합니다. 본 예제에서는 앞서 생성한 validation_generator으로 지정합니다.
- nb_val_samples : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 샘플 수를 지정합니다. 홍 15개의 검증 샘플이 있으므로 15로 지정합니다.

In [ ]:
# 모델 학습시키기
model.fit_generator(
        train_generator,
        samples_per_epoch=45,
        nb_epoch=100,
        validation_data=validation_generator,
        nb_val_samples=15)

---

### 모델 사용하기

학습한 모델을 평가해봅니다. 제네레이터에서 제공되는 샘플로 평가할 때는 evaluate_generator 함수를 사용하고, 예측할 때는 predict_generator 함수를 사용합니다. 

In [ ]:
# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            validation_generator, 
            val_samples = 15)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")

output = model.predict_generator(
            validation_generator, 
            val_samples = 15)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

간단한 모델이고 데이터셋이 적은 데도 불구하고 93.33%라는 높은 정확도를 얻었습니다. 개수로 따지면 검증 샘플 15개 중 1개가 잘 못 분류가 되었네요. Predict 함수는 입력된 이미지에 대해서 모델의 결과를 알려주는 역할을 하는 데, 각 샘플별로 클래스별 확률을 확인할 수 있습니다. 각 열은 다음을 뜻합니다.
- 첫번째 열 : 원일 확률
- 두번째 열 : 사각형일 확률
- 세번째 열 : 삼각형일 확률

확인을 해보니 rectangle020.png 파일이 사격형이 아니라 삼각형으로 판정이 되었습니다. 사각형을 그릴 때는 몰랐었는데, 막상 모델에서 삼각형이라고 얘기하니 다른 사각형이랑 조금 다르게 그린 것 같습니다.

![predict](http://tykimos.github.com/Keras/warehouse/2017-3-8_CNN_Getting_Started_4.png)


---

### 전체 소스

---

### 결론

본 강좌에서는 이미지 분류 문제를 직접 정의해보고 데이터셋도 직접 만들어봤습니다. 이미지 분류 문제에 높은 성능을 보이고 있는 컨볼루션 신경망 모델을 이용하여 직접 만든 데이터셋으로 학습 및 평가를 해보았습니다. 학습 결과는 좋게 나왔지만 이 모델은 한 사람이 그린 것에 대해서만 학습이 되어 있어서 다른 사람에 그린 모양은 잘 분류를 못할 것 같습니다. 이후 강좌에서는 다른 사람이 그린 모양으로 평가해보고 어떻게 모델 성능을 높일 수 있을 지 알아보겠습니다.

그리고 실제 문제에 적용하기 전에 데이터셋을 직접 만들어보거나 좀 더 쉬운 문제로 추상화해서 프로토타이핑 하시는 것을 권장드립니다. 객담도말된 결핵 이미지 판별하는 모델을 만들 때, 결핵 이미지를 바로 사용하지 않고, MNIST의 손글씨 중 '1'과 '7'을 결핵이라고 보고, 나머지는 결핵이 아닌 것으로 학습시켜봤었습니다. 결핵균이 간균 (막대모양)이라 적절한 프로토타이핑이었습니다. 

---

### 같이 보기

* [강좌 목차](https://tykimos.github.io/Keras/lecture/)
* 이전 : [딥러닝 모델 이야기/컨볼루션 신경망 레이어 이야기](https://tykimos.github.io/Keras/2017/01/27/CNN_Layer_Talk/)
* 다음 : [딥러닝 모델 이야기/순환 신경망 레이어 이야기]